In [ ]:
import os

In [ ]:
print(len(os.listdir("rpn_train"))-1)
for i in os.listdir("rpn_train"):
    
    if i != 'bad':
        print(i,len(os.listdir("rpn_train/"+i))/2)

In [ ]:
import cv2
def cv_imread(file_path):
    cv_img = cv2.imdecode(np.fromfile(file_path,dtype=np.uint8),-1)
    return cv_img
def cv_imwrite(file_path,img):
    cv2.imencode('.jpg', img)[1].tofile(file_path)

In [ ]:
#如果文件夹内的数据小于15，则随机做亮度变化数据增强
import shutil
from random import choice
from imgaug import augmenters as iaa
import imgaug as ia
import numpy as np

seq = iaa.Sequential([
            #iaa.Fliplr(1),
            #iaa.Flipud(1),
            iaa.Multiply((0.9, 1.1)),
        ])



for i in os.listdir("rpn_train"):
    if i != 'bad':
        imageList = []
        nameList = []
        k = len(os.listdir("rpn_train/"+i))/2
        print("文件夹内"+str(k)+"张图片")
        #将当前目录下的图片名存在一个list中
        images = []
        for m in os.listdir("rpn_train/"+i):
            if m[-4:] != '.xml':
                images.append(m)
        
        while k < 15:
            #从images中随机取一个名字进行增强
            img_name = 'rpn_train/'+i+'/'+choice(images)
            nameList.append(img_name)
            image = cv_imread(img_name)
            imageList.append(image)
            k+=1
        
        images_aug = seq.augment_images(imageList)
        for n in range(len(images_aug)):
            cv2.imencode('.jpg', images_aug[n])[1].tofile(nameList[n][:-4] + "_augColor_" + str(n) + ".jpg")
            #增强后按名字复制一份xml
            shutil.copyfile(nameList[n][:-4] + '.xml', nameList[n][:-4] + "_augColor_" + str(n) + ".xml")
        print(i+"增强完成")
            
            

In [ ]:
import xml.etree.cElementTree as ET
import sys
import xml.dom.minidom

In [ ]:
#将xml解析出来 将文件名和对应的类别 还有x1,y1,x2,y2放入一个文件夹中
import csv
with open('colorAug_train.csv','a+',newline='') as csv_file:
    writer = csv.writer(csv_file)
    
    for i in os.listdir("rpn_train"):
        #print(i)
        if i != 'bad':
            for j in os.listdir("rpn_train/"+i):
                #print(j)
                if(j[-4:] == '.xml'):
                    DomTree = xml.dom.minidom.parse("rpn_train/"+i+'/'+j)
                    annotation = DomTree.documentElement
                    filename = j[:-4]+'.jpg'
                    #print(j)
                    objectlist = annotation.getElementsByTagName('object')

                    for objects in objectlist:
                        bndbox = objects.getElementsByTagName('bndbox')
                        #print(len(bndbox))
                        for box in bndbox:
                            x1_list = box.getElementsByTagName('xmin')
                            x1 = int(x1_list[0].childNodes[0].data)
                            y1_list = box.getElementsByTagName('ymin')
                            y1 = int(y1_list[0].childNodes[0].data)
                            x2_list = box.getElementsByTagName('xmax')
                            x2 = int(x2_list[0].childNodes[0].data)
                            y2_list = box.getElementsByTagName('ymax')
                            y2 = int(y2_list[0].childNodes[0].data)
                            #print(x1,y1,x2,y2)
                            writer.writerow([filename,'bad',x1,y1,x2,y2])
    
                        
                
    

In [ ]:
#所有图片复制到rpn_train/bad中等待增强
for i in os.listdir("rpn_train"):
    if i != 'bad':
        for j in os.listdir("rpn_train" + '/' + i):
            if j[-4:]!='.xml':
                shutil.copy("rpn_train" + '/' + i + '/' + j,"rpn_train/bad/")
                

In [ ]:
img_bbox_dict = {}
with open("colorAug_train.csv","r+") as csv_file:
    for line in csv_file.readlines():
        img_name,category,x1,y1,x2,y2 = line[:-1].split(',')
        if img_name not in img_bbox_dict:
            img_bbox_dict[img_name] = []
        img_bbox_dict[img_name].append([x1,y1,x2,y2])
        
        

In [ ]:
img_bbox_dict

In [ ]:
#左右翻转
def Fliplr(img):
    seq = iaa.Sequential([
            iaa.Fliplr(1),
        ])
    imageList=[img]
    images_aug = seq.augment_images(imageList)
    return images_aug[0]

#上下翻转
def Flipud(img):
    seq = iaa.Sequential([
            iaa.Flipud(1),
        ])
    imageList=[img]
    images_aug = seq.augment_images(imageList)
    return images_aug[0]

def rotationPi(img):
    seq = iaa.Sequential([
            iaa.Affine(
                rotate = (180, 180)
            )
        ])
    imageList=[img]
    images_aug = seq.augment_images(imageList)
    return images_aug[0]

In [ ]:
import transform.transform as tr

In [ ]:
abc=1
with open("color_flip_rotate_Aug_train.csv",'a',newline='') as csv_file:
    writer = csv.writer(csv_file)
    for i in os.listdir("rpn_train/bad"):
        #对每一个图片进行左右翻转，上下翻转，旋转180度三种数据增强，并且得到新的bbox值
        print(abc)
        img = cv_imread("rpn_train/bad/" + i)
        #左右翻转
        imglr = Fliplr(img)
        for bbox in img_bbox_dict[i]:
            writer.writerow([i,'bad',bbox[0],bbox[1],bbox[2],bbox[3]])
            bbox_new = tr.transform_aabb(transform=tr.change_transform_origin(transform=tr.random_flip(flip_x_chance=1,flip_y_chance=0),center=(1280,960)),aabb=[int(bbox[0]),int(bbox[1]),int(bbox[2]),int(bbox[3])])
            #print(bbox_new)
            writer.writerow([i[:-4] + 'fliplr.jpg','bad',bbox_new[0],bbox_new[1],bbox_new[2],bbox_new[3]])
        cv_imwrite(file_path='rpn_train/bad_aug/'+ i[:-4] + 'fliplr.jpg',img=imglr)
        #上下翻转
        imgud = Flipud(img)
        for bbox in img_bbox_dict[i]:
            bbox_new = tr.transform_aabb(transform=tr.change_transform_origin(transform=tr.random_flip(flip_x_chance=0,flip_y_chance=1),center=(1280,960)),aabb=[int(bbox[0]),int(bbox[1]),int(bbox[2]),int(bbox[3])])
            writer.writerow([i[:-4] + 'flipud.jpg','bad',bbox_new[0],bbox_new[1],bbox_new[2],bbox_new[3]])
        cv_imwrite(file_path='rpn_train/bad_aug/'+ i[:-4] + 'flipud.jpg',img=imgud)
        #旋转180度
        imgrotate = rotationPi(img)
        for bbox in img_bbox_dict[i]:
            bbox_new = tr.transform_aabb(transform=tr.change_transform_origin(transform=tr.random_rotation(max=np.pi,min=np.pi),center=(1280,960)),aabb=[int(bbox[0]),int(bbox[1]),int(bbox[2]),int(bbox[3])])
            writer.writerow([i[:-4] + 'rotatePi.jpg','bad',int(bbox_new[0]),int(bbox_new[1]),int(bbox_new[2]),int(bbox_new[3])])
        cv_imwrite(file_path='rpn_train/bad_aug/'+ i[:-4] + 'rotatePi.jpg',img=imgrotate)
        abc += 1
    
    

In [ ]:
#对于验证集
#将xml解析出来 将文件名和对应的类别 还有x1,y1,x2,y2放入一个文件夹中
import csv
with open('val.csv','a+',newline='') as csv_file:
    writer = csv.writer(csv_file)
    
    for i in os.listdir("val"):
        #print(i)
        if i == '瑕疵xml':
            for j in os.listdir("val/瑕疵xml"):
                #print(j)
                if(j[-4:] == '.xml'):
                    DomTree = xml.dom.minidom.parse("val/瑕疵xml/"+j)
                    annotation = DomTree.documentElement
                    filename = j[:-4]+'.jpg'
                    #print(j)
                    objectlist = annotation.getElementsByTagName('object')

                    for objects in objectlist:
                        bndbox = objects.getElementsByTagName('bndbox')
                        #print(len(bndbox))
                        for box in bndbox:
                            x1_list = box.getElementsByTagName('xmin')
                            x1 = int(x1_list[0].childNodes[0].data)
                            y1_list = box.getElementsByTagName('ymin')
                            y1 = int(y1_list[0].childNodes[0].data)
                            x2_list = box.getElementsByTagName('xmax')
                            x2 = int(x2_list[0].childNodes[0].data)
                            y2_list = box.getElementsByTagName('ymax')
                            y2 = int(y2_list[0].childNodes[0].data)
                            #print(x1,y1,x2,y2)
                            writer.writerow([filename,'bad',x1,y1,x2,y2])

